In [5]:
%load_ext autoreload
%autoreload 2

import smtplib as smtp 
import ssl 
import os

import queue
import multiprocessing

from functions import *
from master_functions import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mail

In [2]:
mail = 'drainingytb@gmail.com'
pswd = 'YoDH@ck3r'

In [3]:
ssl_cntxt = ssl.create_default_context()

In [14]:
smtp.SMTP_SSL()


with smtplib.SMTP_SSL("smtp.gmail.com", port=465, context=ssl_cntxt) as server:
    server.login(mail, pswd)
    server.sendmail(from_addr=mail,
                    to_addrs='xabier.rubiato@epfl.ch',
                    msg='Finish Process')

## Download

In [16]:
from multiprocessing import Process, Queue, current_process, freeze_support

In [2]:
URL = 'https://www.youtube.com/watch?v=2S7Yak00e6E&list=PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b'
PLAYLIST = 'test'
N_ITEMS = 6 
PTH_PLYLST = pth_plylst(name=PLAYLIST)

In [4]:
print('Start download of playlist : {}\n'.format(PLAYLIST))
yt_dl(url= URL,
      playlist=PLAYLIST,
      n_items= N_ITEMS)
print('Finish download of playlist : {}\n'.format(PLAYLIST))

Start download of playlist : test

[youtube:playlist] Downloading playlist PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b - add --no-playlist to just download video 2S7Yak00e6E
[youtube:playlist] PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b: Downloading webpage
[download] Downloading playlist: Alignments as vines
[youtube:playlist] playlist Alignments as vines: Downloading 6 videos
[download] Downloading video 1 of 6
[youtube] 2S7Yak00e6E: Downloading webpage
[youtube] 2S7Yak00e6E: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/2S7Yak00e6E/data/2S7Yak00e6E_1920x1080.info.json


[download] videos/test/2S7Yak00e6E/data/2S7Yak00e6E_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 2 of 6
[youtube] qoqZ9T5O29g: Downloading webpage
[youtube] qoqZ9T5O29g: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/qoqZ9T5O29g/data/qoqZ9T5O29g_1920x1080.info.json


[download] videos/test/qoqZ9T5O29g/data/qoqZ9T5O29g_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 3 of 6
[youtube] IMVmevu6mE4: Downloading webpage
[youtube] IMVmevu6mE4: Downloading embed webpage
[youtube] IMVmevu6mE4: Refetching age-gated info webpage
[info] Writing video description metadata as JSON to: videos/test/IMVmevu6mE4/data/IMVmevu6mE4_1920x1080.info.json
[download] videos/test/IMVmevu6mE4/data/IMVmevu6mE4_1920x1080.mp4 has already been downloaded and merged
[download] Downloading video 4 of 6
[youtube] MNMYYEdJoC0: Downloading webpage
[youtube] MNMYYEdJoC0: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/MNMYYEdJoC0/data/MNMYYEdJoC0_1920x1080.info.json
[download] videos/test/MNMYYEdJoC0/data/MNMYYEdJoC0_1920x1080.mp4 has already been downloaded and merged
[download] Downloading video 5 of 6
[youtube] -Ia18bqO-9w: Downloading webpage
[youtube] -Ia18bqO-9w: Downloading video info webpage
[inf

[download] videos/test/-Ia18bqO-9w/data/-Ia18bqO-9w_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 6 of 6
[youtube] yd7d51AgSso: Downloading webpage
[youtube] yd7d51AgSso: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.info.json
[download] Destination: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f248.webm
[download] 100% of 43.28MiB in 00:1157MiB/s ETA 00:005
[download] Destination: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f251.webm
[download] 100% of 4.08MiB in 00:0168MiB/s ETA 00:004
[ffmpeg] Merging formats into "videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.webm"
Deleting original file videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f248.webm (pass -k to keep)
Deleting original file videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f251.webm (pass -k to keep)
[download] Finished downloading playlist: Alignments as vines
Finish downloa

In [8]:
l1 = [None]*30
for i in range(len(l1)): 
    l1[i] = [None]*30
    
k = 0 
for j in range(len(l1)):
    for i  in range(len(l1[j])): 
        k +=1
        print(k)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [78]:
for i in range(10):
    for j in range(i+1):
        k+=1

In [80]:
k

55

## Multiprocessing

In [125]:
%load_ext autoreload
%autoreload 2

import os
from functions import *
from functions import pth_plylst
from master_functions import iter0, sfm_pipe
import multiprocessing 
import queue 
from multiprocessing import Process, Queue, cpu_count

In [118]:
class Worker: 
    
    def __init__(task_queue):
        self.q = task_queue
    
    def iter0(pth_vid):
        """Performs iter0() and enqueues an sfm_pipe process with the generated sets."""
        pths = iter0(pth_vid)
        pth_sets = [os.path.join(pth, el) for el in os.listdir(pths)]

        # Enqueue next tasks
        for pth_set in pth_sets:  
            q.put(sfm_pipe, pth_set)
    
    def sfm_pipe(pth_vid):
        pass

In [ ]:
def worker_exec(task_queue):
    
    while not task_queue.empty():
        
        worker = Worker(task_queue)
        task = task_queue.get()
        print(task[0], '\n')
        print(task[1], '\n')
        if task[0] == 'iter0':
            worker.iter0(task[1])
            
        elif task[0] == 'sfm_pipe':
            worker.sfm_pipe(task[1])

### Launch processes: 

In [ ]:
NUMBER_PROCESSES = cpu_count()
PLYLST = 'test'
path_plylst = pth_plylst(PLYLST)

In [ ]:
# Create Queue 
task_queue = Queue()

In [7]:
# Fille queue with the iter0 jobs 
for v_id in os.listdir(path_plylst): 
    pth = pth_vid(v_id, PLYLST)
    task_queue.put(('Qiter0', pth))

In [8]:
for i in range(NUMBER_PROCESSES):
    p = Process(target=worker_exec, args=[task_queue,])
    p.start()

Qiter0 

/Users/xrubiato/DHLab/Draining-Youtube/videos/test/qoqZ9T5O29g 
Qiter0 


in iter0()
/Users/xrubiato/DHLab/Draining-Youtube/videos/test/MNMYYEdJoC0 

Qiter0 
in iter0()

/Users/xrubiato/DHLab/Draining-Youtube/videos/test/IMVmevu6mE4 
Qiter0 


in iter0()
/Users/xrubiato/DHLab/Draining-Youtube/videos/test/-Ia18bqO-9w 
Qiter0 


in iter0()
/Users/xrubiato/DHLab/Draining-Youtube/videos/test/yd7d51AgSso 

Qiter0 
in iter0()

/Users/xrubiato/DHLab/Draining-Youtube/videos/test/2S7Yak00e6E 

in iter0()


## Tests

In [46]:
from functions import * 
from master_functions import iter0 

In [42]:
url = 'https://www.youtube.com/watch?v=ftlvreFtA2A0'
v_id = url_to_id(url)
yt_dl(url)
path_vid = pth_vid(v_id)

[youtube] ftlvreFtA2A: Downloading webpage
[youtube] ftlvreFtA2A: Downloading video info webpage
[youtube] Downloading just video ftlvreFtA2A because of --no-playlist
[info] Writing video description metadata as JSON to: videos/ftlvreFtA2A/data/ftlvreFtA2A_3840x2160.info.json
[download] videos/ftlvreFtA2A/data/ftlvreFtA2A_3840x2160.webm has already been downloaded and merged


In [7]:
from functions import *
from master_functions import iter0

url = 'https://www.youtube.com/watch?v=ftlvreFtA2A0'
yt_dl(url)

iter0(path_vid=pth_vid(url_to_id(url)), sample=True)

/Users/xrubiato/DHLab/Draining-Youtube/videos/ftlvreFtA2A0/frames


FileNotFoundError: [Errno 2] No such file or directory: '/Users/xrubiato/DHLab/Draining-Youtube/videos/ftlvreFtA2A0/iter0/features/matches.f.txt'